# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [15]:
!pip install citipy

# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from citipy import citipy 
from config import g_key

hotel_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'

# Hotel Parameters
target_radius = 5000
target_type = 'hotel'
target_search = 'hotel'

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [16]:
# Reference WeatherPy city data file for this assignment
weatherpy_data = pd.read_csv('../Output/clean_citydata_flat.csv')
city_data_df = pd.DataFrame(weatherpy_data)




### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [17]:
#Configure gmaps
gmaps.configure(api_key=g_key)


In [18]:
# REMEMBER to handle NaN values
city_data_df = city_data_df.dropna()
temperature = city_data_df["temp_max"].astype(float)
locations = city_data_df[["latitude", "longitude"]].astype(float)
cloudiness = city_data_df["cloudiness"].astype(float)

# Create a poverty Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=temperature, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [19]:
# My ideal weather conditions:  max_temp betwen 70-80 degrees, wind_speed < 10 mph, cloudiness = 25%

my_vacation_df = city_data_df[
                (city_data_df['temp_min']>=65) &
                (city_data_df['temp_max']<=80) &
                (city_data_df['wind_speed']<=12) &
                (city_data_df['cloudiness']<=25)]

print(len(my_vacation_df))
my_vacation_df = my_vacation_df.reset_index(drop=True)
my_vacation_df.head()


51


,id,name,country,latitude,longitude,temp,temp_max,temp_min,humidity,cloudiness,wind_speed
0,4030556,Rikitea,PF,-23.12,-134.97,74.08,74.08,74.08,75.0,24,9.31
1,876177,Luau,AO,-10.71,22.22,65.73,65.73,65.73,73.0,4,2.93
2,2324857,Pindiga,NG,9.98,10.93,71.42,71.42,71.42,77.0,0,3.47
3,2389804,Baoro,CF,5.67,15.97,67.84,67.84,67.84,80.0,6,1.32
4,934479,Grand Gaube,MU,-20.01,57.66,75.00,75.00,75.00,71.0,0,7.00


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [20]:
my_vacation_df.loc[:,"coordinates"] = my_vacation_df.loc[:,"latitude"].astype(str) +"," + my_vacation_df.loc[:,"longitude"].astype(str)

In [21]:
nearby_hotel = []

for location in my_vacation_df['coordinates']:
    
    gmaps_url = f'{hotel_url}location={location}&radius={target_radius}&type={target_type}&keyword={target_search}&key={g_key}'
    hotel_json = requests.get(gmaps_url).json()

    try:
        nearby_hotel.append(hotel_json['results'][0]['name'])
        
    except:
        nearby_hotel.append("None nearby")



In [22]:
# Prep dataframe to work in fixed logic in next command box
my_vacation_df["Hotel Name"] = nearby_hotel

my_vacation_df = my_vacation_df.rename(columns = {'name':'City',
                                                  'country': 'Country',
                                                  'latitude': 'Lat',
                                                  'longitude': 'Lng'})

my_vacation_df.head()

,id,City,Country,Lat,Lng,temp,temp_max,temp_min,humidity,cloudiness,wind_speed,coordinates,Hotel Name
0,4030556,Rikitea,PF,-23.12,-134.97,74.08,74.08,74.08,75.0,24,9.31,"-23.12,-134.97",Pension Maro'i
1,876177,Luau,AO,-10.71,22.22,65.73,65.73,65.73,73.0,4,2.93,"-10.71,22.22",Kimbo Hotel
2,2324857,Pindiga,NG,9.98,10.93,71.42,71.42,71.42,77.0,0,3.47,"9.98,10.93",Accommodation Center
3,2389804,Baoro,CF,5.67,15.97,67.84,67.84,67.84,80.0,6,1.32,"5.67,15.97",None nearby
4,934479,Grand Gaube,MU,-20.01,57.66,75.00,75.00,75.00,71.0,0,7.00,"-20.01,57.66",LUX Grand Gaube Resort & Villas


In [23]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in my_vacation_df.iterrows()]
locations = my_vacation_df[["Lat", "Lng"]]

In [24]:
# Add marker layer ontop of heat map and display and display
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(heat_layer)

fig.add_layer(markers)

fig


Figure(layout=FigureLayout(height='420px'))